In [34]:
%matplotlib inline

In [35]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
df = pd.read_csv('indeed_job_dataset_short.csv')

In [36]:
df.shape

(5715, 12)

In [37]:
list(df.head())

['Salary1',
 'Salary2',
 'Job_Type',
 'Skill',
 'No_of_Skills',
 'No_of_Reviews',
 'No_of_Stars',
 'Date_Since_Posted',
 'Location',
 'Revenue',
 'Employees',
 'Industry']

In [38]:
cat_list=['Salary1',
 'Location',
 'Revenue',
 'Employees',
 'Industry']

In [39]:
df_noskill = df.drop(['Skill'], axis=1)

In [40]:
df_skill = df['Skill'].str.get_dummies(sep=', ')
df_skill = df_skill.add_prefix('skill_')

In [41]:
df_skill.head()

,skill_ Project / Program Management,"skill_""Drivers License""",skill_.Net,skill_3G,skill_4G/LTE,skill_ACH,skill_ADAS,skill_AI,skill_AJAX,skill_APICS Certification,skill_AWS,skill_Accounting,skill_Acquisition Strategy,skill_Active Directory,skill_Adobe Acrobat,skill_Adobe Creative Suite,skill_Adobe InDesign,skill_Adobe Marketing,skill_Adobe Photoshop,skill_Affiliate Marketing,skill_Agriculture,skill_Akka,skill_Alteryx,skill_Analysis Skills,skill_Android,skill_Angular,skill_Ansible,skill_Ant,skill_Apache,skill_Application Support,skill_ArcGIS,skill_Attribution Modeling,skill_Audits,skill_Aviation Experience,skill_Azure,skill_B2B Sales,skill_BC/BE,skill_BGP,skill_Basic Computer Skills,skill_Big Data,skill_Bilingual,skill_Bioinformatics,skill_Biostatistics,skill_Bootstrap,skill_Build Automation,skill_Business Analysis,skill_Business Intelligence,skill_Business Management,skill_Business Process Reengineering,skill_C#,skill_C/C++,skill_CAD,skill_CAD Software,skill_CCDA,skill_CCNA,skill_CCNA Security,skill_CCNP,skill_CDISC Standards,skill_CEH,skill_CFE,skill_CGMP,skill_CI,skill_CI/CD,skill_CISM,skill_CISSP,skill_CPA,skill_CPHQ,skill_CPT Coding,skill_CSS,skill_CSSLP,skill_Calculus,skill_Calibration,skill_Cash Flow Analysis,skill_Cassandra,skill_Cell Biology,skill_CentOS,skill_Cerner,skill_Certified Compensation Professional,skill_Certified Compliance And Ethics Professional,skill_Certified Information Systems Auditor,skill_Certified Internal Auditor,skill_Certified Management Accountant,skill_Certified Project Manager,skill_Certified Scrum Master,skill_Change Management,skill_Chartered Financial Analyst,skill_Chef,skill_Cisco Routers,skill_Clean Driving Record,skill_Clinical Research,skill_Clinical Trials,skill_Clojure,skill_Cloud Computing,skill_Coaching Experience,skill_CompTIA Network+,skill_CompTIA Security+,skill_Compliance Management,skill_Computer Graphics,skill_Conflict Management,skill_Confluence,skill_Content Management Systems,skill_Continuous Improvement,skill_Control-M,skill_Conversion Optimization,skill_Cordova,skill_Crystal Reports,skill_Customer Segmentation,skill_Customer Service,skill_Cybersecurity,skill_DB2,skill_DHCP,skill_DNS,skill_Data Analysis,skill_Data Management,skill_Data Mining,skill_Data Science,skill_Data Warehouse,skill_Database Administration,skill_Database Marketing,skill_Design Controls,skill_Design Experience,skill_Design Patterns,skill_Digital Marketing,skill_Distributed Control Systems,skill_DoD 8570,skill_Docker,skill_Doubleclick,skill_Drupal,skill_DynamoDB,skill_E-Commerce,skill_ERP Systems,skill_Eclipse,skill_Elasticsearch,skill_Ellucian,skill_Email Marketing,skill_Embedded Software,skill_Ember,skill_Employee Evaluation,skill_Enrolled Agent,skill_Epic,skill_Epic Certification,skill_Epidemiology,skill_Erlang,skill_Excel,skill_Experimental Design,skill_FINRA License,skill_FIPS,skill_FISMA,skill_FPGA,skill_FTP,skill_Facebook Advertising,skill_FedRAMP,skill_FileMaker,skill_Financial Accounting,skill_Financial Modeling,skill_Firewall,skill_Fortran,skill_Front-End Development,skill_GAAP,skill_GIS,skill_GPEN,skill_GSEC,skill_Genetics,skill_Git,skill_Go,skill_Google AdWords,skill_Google Analytics,skill_Google Cloud Platform,skill_Google Search Console,skill_Google Suite,skill_Grant Writing,skill_GraphQL,skill_HBase,skill_HDI Certification,skill_HP ALM,skill_HTML5,skill_HTTPS,skill_Hadoop,skill_Haskell,skill_Heroku,skill_Hibernate,skill_Hive,skill_HubSpot,skill_Hyperion,skill_IAT,skill_IAT Level I,skill_IAT Level II,skill_ICD Coding,skill_ICD-10,skill_ICD-9,skill_ICU,skill_IDS,skill_IFRS,skill_IIS,skill_IPsec,skill_ISO 9001,skill_IT Experience,skill_ITIL,skill_IVR,skill_IaaS,skill_Identity And Access Management,skill_Image Processing,skill_Immunoassays,skill_InVision,skill_Informatica,skill_Information Security,skill_Integrated Library System,skill_Intelligence Community Experience,skill_Intelligence Experience,skill_Internal Audits,skill_JBoss,skill_JD Edwards,skill_JNCIA,skill_JSON,skill_Java,skill_J

In [42]:
df_noskill.head()

,Salary1,Salary2,Job_Type,No_of_Skills,No_of_Reviews,No_of_Stars,Date_Since_Posted,Location,Revenue,Employees,Industry
0,less than80000,80000,data_scientist,2,3301.0,3.3,1.0,MO,more than $10B,"10,000+",Health Care
1,less than80000,80000,data_scientist,5,NaN,NaN,15.0,TX,NaN,NaN,NaN
2,less than80000,80000,data_scientist,9,62.0,3.5,1.0,OR,NaN,NaN,NaN
3,less than80000,80000,data_scientist,1,158.0,4.3,30.0,DC,NaN,NaN,Government
4,less than80000,80000,data_scientist,7,495.0,4.1,30.0,TX,NaN,"less than 10,000",Banks and Financial Services


In [43]:
df_noskill_dum = pd.get_dummies(df_noskill,prefix=['Sal_','Type_','loc_', 'Rev_','Emp_','Indu'])

In [44]:
df_new = pd.concat([df_noskill_dum,df_skill], axis=1)

In [45]:
df_new = pd.concat([df_noskill_dum,df_skill], axis=1)

In [46]:
df_new2 = pd.concat([df['Salary1'].to_frame(),df_new],axis=1)

In [47]:
df_new2.dropna(inplace=True)

In [48]:
#df_new.to_csv(r'df_new.csv')

In [49]:
df_new.columns[5:11]

Index(['Sal__100000-119999', 'Sal__120000-139999', 'Sal__140000-159999',
       'Sal__80000-99999', 'Sal__less than80000', 'Sal__more than160000'],
      dtype='object')

In [50]:
feature_name = []
for col in list(df_new.columns):
    if col not in list(df_new.columns)[5:11]:
        feature_name.append(col)
    else:
        continue

In [51]:
feature_name.remove('Salary2')

In [52]:
df_new2['Salary1'].unique()

array(['less than80000', '80000-99999', '100000-119999', '120000-139999',
       '140000-159999', 'more than160000'], dtype=object)

In [53]:
X = df_new2[feature_name]
y = df_new2['Salary2']


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [54]:
from sklearn.ensemble import RandomForestRegressor

In [55]:
def feature_sampling_scores (max_features_list):
    feature_sampling_scores_list = []
    for max_feature in max_features_list:
        rfr = RandomForestRegressor(max_features=max_feature, random_state=1,n_estimators=10)
        rfr.fit(X_train, y_train)
        score = (max_feature, rfr.score(X_val,y_val))
        feature_sampling_scores_list.append(score)
    return feature_sampling_scores_list

In [56]:
feature_sampling_scores([0.5,'log2','sqrt',None])

[(0.5, 0.5448145075391524),
 ('log2', 0.5261852824656237),
 ('sqrt', 0.5503411717079542),
 (None, 0.5578159467731716)]

### Feature Importances

In [57]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 6, max_features=0.5, min_samples_leaf=8).fit(X_train, y_train)

In [58]:
rfr.score(X_val, y_val)

0.5193807495422667

In [59]:
importance_numbers = rfr.feature_importances_
importance_list = importance_numbers.tolist()
importances = np.vstack((feature_name, importance_list))
importances = pd.Series(importance_list, index = feature_name)
importances.sort_values(ascending=False)

Type__data_analyst                                                         0.278158
loc__CA                                                                    0.081270
skill_Python                                                               0.080659
skill_Hadoop                                                               0.076394
No_of_Reviews                                                              0.058187
Type__data_scientist                                                       0.053609
No_of_Stars                                                                0.046897
skill_Machine Learning                                                     0.035431
skill_Spark                                                                0.031511
loc__NY                                                                    0.029563
No_of_Skills                                                               0.029329
Date_Since_Posted                                                          0

In [78]:
import xgboost as xgb

X = df_new2[feature_name]
y = df_new2['Salary1']

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=200, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy))


accuracy: 0.495218
